In [3]:
# Import dependencies
import requests
import os
import pprint
from datetime import datetime,timedelta
import json
import pandas as pd
from dateutil.parser import parse
import itertools
from itertools import permutations
import tqdm 
from tqdm import tqdm


In [4]:
# Set params for query
pax = input(' # of adults will be traveling?')
children = input(' # Children traveling with you?')
p0 = input(' Starting place') 
startdate = input('Trip start date (mm/dd/yyyy)')
destinations = input('List the cities do you want to travel to? And how many days per city? ex(London,4,Madrid,3)')
destinations.replace(" ", "")
destinations1 = list(destinations.split(','))




 # of adults will be traveling?1
 # Children traveling with you?0
 Starting placemiami
Trip start date (mm/dd/yyyy)12/01/2019
List the cities do you want to travel to? And how many days per city? ex(London,4,Madrid,3)madrid,3,paris,4,nyc,6


In [108]:
p1=(str(destinations1[0]),int(destinations1[1]))
p2 = (str(destinations1[2]), int(destinations1[3]))
p3 = (str(destinations1[4]), int(destinations1[5]))


def changeorder(res,p1,p2,p3,startdate,p0,children,pax):
    

    start_date = parse(startdate)
    days_p1=p1[1]
    days_p2=p2[1]
    days_p3=p3[1]
    start_p1 = start_date + timedelta(days=days_p1)
    start_p2 = start_p1 + timedelta(days=days_p2)
    start_p3 = start_p2 + timedelta(days=days_p3)
    url = "https://apidojo-hipmunk-v1.p.rapidapi.com/flights/create-session"

    querystring = {"from1":p1[0],"from2":p2[0],"from3":p3[0],"to1":p2[0],"to2":p3[0],"to3":p0,"date1":start_p1,"date2":start_p2,"date3":start_p3,"infants_lap":"0","children":children,"seniors":"0","country":"US","from0":p0,"to0":p1[0],"date0":startdate,"pax":pax,"cabin":"Coach"}
    headers = {
        'x-rapidapi-host': "apidojo-hipmunk-v1.p.rapidapi.com",
        'x-rapidapi-key': "c36e6ce6fcmsh7fe55ace5e9f40fp10fdc1jsn690a20bbc0ef"
        }

    response = requests.request("GET", url, headers=headers, params=querystring).json()

    #does stuff with response
    idnums=[]
    agonies=[]
    prices=[]
    routingid=[]
    

    for key,value in response['itins'].items():
        idnum=key
        agony=value['agony']
        price=value['price']
        routing_id=value['routing_idens']
        idnums.append(idnum)
        agonies.append(agony)
        prices.append(price)
        routingid.append(routing_id)
    df=pd.DataFrame()
    df['price']=prices
    df['agonies']=agonies
    df['id']=idnums
    df['routing']=routingid
    
    
    
    


    # Print DF with values
    df = df.sort_values(['price','agonies'], ascending=[True,True])
    df = df[:1]

    res.append([(p1,p2,p3),df])



    #p1,p2,p3 = p2,p3,p1
    return res
    
    

In [109]:
alist = []
listA = [p1, p2, p3]
perm = itertools.permutations(listA) 
  
for i in list(perm): 
    alist.append(i)
alist
    
    
    

[(('madrid', 3), ('paris', 4), ('nyc', 6)),
 (('madrid', 3), ('nyc', 6), ('paris', 4)),
 (('paris', 4), ('madrid', 3), ('nyc', 6)),
 (('paris', 4), ('nyc', 6), ('madrid', 3)),
 (('nyc', 6), ('madrid', 3), ('paris', 4)),
 (('nyc', 6), ('paris', 4), ('madrid', 3))]

In [110]:
print('Sorry, this may take a while...')
#res=[]
#for i in tqdm(alist):
#    p1,p2,p3=i[0],i[1],i[2]
#    res=changeorder(res,p1,p2,p3,startdate,p0,children,pax)
    
res=[]
for i in tqdm(alist):
    p1,p2,p3=i[0],i[1],i[2]
    res=changeorder(res,p1,p2,p3,startdate,p0,children,pax)    
    
print('Search 100% completed')   
res




















  0%|          | 0/6 [00:00<?, ?it/s]

Sorry, this may take a while...





















 17%|█▋        | 1/6 [00:21<01:46, 21.35s/it]


















 33%|███▎      | 2/6 [00:42<01:25, 21.40s/it]


















 50%|█████     | 3/6 [01:03<01:03, 21.02s/it]


















 67%|██████▋   | 4/6 [01:23<00:41, 20.72s/it]


















 83%|████████▎ | 5/6 [01:37<00:18, 18.81s/it]


















100%|██████████| 6/6 [01:56<00:00, 19.41s/it]

Search 100% completed


[[(('madrid', 3), ('paris', 4), ('nyc', 6)),
        price      agonies                                id  \
  5667  732.0  1883.012731  ddb1d17654b4e36da9cff9055afe106c   
  
                                                  routing  
  5667  [458dfcf2fb8507a7fc373035c2f493c1, a97c203f534...  ],
 [(('madrid', 3), ('nyc', 6), ('paris', 4)),
         price      agonies                                id  \
  6523  1436.0  3404.276434  4fa59484b20be4eed1e49f71ca660a33   
  
                                                  routing  
  6523  [458dfcf2fb8507a7fc373035c2f493c1, d37e643ab72...  ],
 [(('paris', 4), ('madrid', 3), ('nyc', 6)),
        price      agonies                                id  \
  3675  750.0  1874.061677  ba8d60e9031abe641c8fcb5209cc0c5f   
  
                                                  routing  
  3675  [75f75c75221f6185973ac7818bead59c, 56b60da2af7...  ],
 [(('paris', 4), ('nyc', 6), ('madrid', 3)),
        price      agonies                                i

In [111]:
df1=pd.DataFrame(res[0][1])
df2=pd.DataFrame(res[1][1])
df3=pd.DataFrame(res[2][1])
df4=pd.DataFrame(res[3][1])
df5=pd.DataFrame(res[4][1])
df6=pd.DataFrame(res[5][1])


In [124]:
df1

,price,agonies,id,routing
5667,732.0,1883.012731,ddb1d17654b4e36da9cff9055afe106c,"[458dfcf2fb8507a7fc373035c2f493c1, a97c203f534..."


In [136]:
#Joining all results into a single DF:

#Option 1
first_choice = df1['routing'].astype(str)
df1new = first_choice.str.split(", ",expand=True)
firstchoice_merged = df1.merge(df1new, how='outer', left_index=True, right_index=True)
firstchoice_merged = firstchoice_merged.drop(columns=['routing'])

#Option 2
second_choice = df2['routing'].astype(str)
df2new = second_choice.str.split(", ",expand=True)
secondchoice_merged = df2.merge(df2new, how='outer', left_index=True, right_index=True)
secondchoice_merged = secondchoice_merged.drop(columns=['routing'])

#Option 3
third_choice = df3['routing'].astype(str)
df3new = third_choice.str.split(", ",expand=True)
thirdchoice_merged = df3.merge(df3new, how='outer', left_index=True, right_index=True)
thirdchoice_merged = thirdchoice_merged.drop(columns=['routing'])

#Option 4
fourth_choice = df4['routing'].astype(str)
df4new = fourth_choice.str.split(", ",expand=True)
fourthchoice_merged = df4.merge(df4new, how='outer', left_index=True, right_index=True)
fourthchoice_merged = fourthchoice_merged.drop(columns=['routing'])

#Option 5
fifth_choice = df5['routing'].astype(str)
df5new = fifth_choice.str.split(", ",expand=True)
fifthchoice_merged = df5.merge(df5new, how='outer', left_index=True, right_index=True)
fifthchoice_merged = fifthchoice_merged.drop(columns=['routing'])

#Option 6
sixth_choice = df6['routing'].astype(str)
df6new = sixth_choice.str.split(", ",expand=True)
sixthchoice_merged = df6.merge(df6new, how='outer', left_index=True, right_index=True)
sixthchoice_merged = sixthchoice_merged.drop(columns=['routing'])

bigdata = [firstchoice_merged,secondchoice_merged,thirdchoice_merged,fourthchoice_merged,fifthchoice_merged,sixthchoice_merged]
result = pd.concat(bigdata)
#result_df = result.sort_values(['price','agonies'], ascending=[True,True])

In [137]:
resultfull_df=result_df


In [138]:
resultuse_df = result.drop(columns=[0,1,2,3])
resultuse_df


,price,agonies,id
5667,732.0,1883.012731,ddb1d17654b4e36da9cff9055afe106c
6523,1436.0,3404.276434,4fa59484b20be4eed1e49f71ca660a33
3675,750.0,1874.061677,ba8d60e9031abe641c8fcb5209cc0c5f
525,1429.0,3224.287875,752515de0857c63004ee9879218ac388
2471,1126.0,2637.771610,8cf29a7fddcb556e4205288d2b779e45
3712,972.0,2218.056889,07974516c323a35caacc39bda66d0052


In [162]:
xlist1=[]
ylist1=[]
xlist2=[]
ylist2=[]
xlist3=[]
ylist3=[]

for x,y,z in alist:
    d1,d2 = x
    xlist1.append(d1)
    ylist1.append(d2)
    d3,d4 = y
    xlist2.append(d3)
    ylist2.append(d4)
    d5,d6 = z
    xlist3.append(d5)
    ylist3.append(d6)
print(xlist1)
print(ylist1)



['madrid', 'madrid', 'paris', 'paris', 'nyc', 'nyc']
[3, 3, 4, 4, 6, 6]


In [157]:
alist

[(('madrid', 3), ('paris', 4), ('nyc', 6)),
 (('madrid', 3), ('nyc', 6), ('paris', 4)),
 (('paris', 4), ('madrid', 3), ('nyc', 6)),
 (('paris', 4), ('nyc', 6), ('madrid', 3)),
 (('nyc', 6), ('madrid', 3), ('paris', 4)),
 (('nyc', 6), ('paris', 4), ('madrid', 3))]

In [152]:
alist_df = pd.DataFrame(alist)
alist_df
alist_df

,0,1,2
0,"(madrid, 3)","(paris, 4)","(nyc, 6)"
1,"(madrid, 3)","(nyc, 6)","(paris, 4)"
2,"(paris, 4)","(madrid, 3)","(nyc, 6)"
3,"(paris, 4)","(nyc, 6)","(madrid, 3)"
4,"(nyc, 6)","(madrid, 3)","(paris, 4)"
5,"(nyc, 6)","(paris, 4)","(madrid, 3)"


In [197]:
resultuse_df['Starting Point'] = p0
resultuse_df['Starting Date'] = startdate
resultuse_df['First_Stop'] = xlist1
resultuse_df['Lenght_of_stay_1'] = ylist1
resultuse_df['Second_Stop'] = xlist2
resultuse_df['Lenght_of_stay_2'] = ylist2
resultuse_df['Third_Stop'] = xlist3
resultuse_df['Lenght=_of_stay_3'] = ylist3
resultuse_df['End_Point'] = p0
#resultuse_df = resultuse_df.drop[['Starting_Point','Starting_Date']]
df_full = resultuse_df[['price','agonies','id','Starting_Point','Starting_Date','First_Stop','Lenght_of_stay_1','Second_Stop','Lenght_of_stay_2','Third_Stop','Lenght=_of_stay_3','End_Point']]

In [202]:
df_fullnew = df_full.rename(columns={"id":'Itinerary_#'})
df_fullnew=df_fullnew.sort_values(['price','agonies'], ascending=[True,True])

In [203]:
df_fullnew

,price,agonies,Itinerary_#,Starting_Point,Starting_Date,First_Stop,Lenght_of_stay_1,Second_Stop,Lenght_of_stay_2,Third_Stop,Lenght=_of_stay_3,End_Point
5667,732.0,1883.012731,ddb1d17654b4e36da9cff9055afe106c,miami,12/01/2019,madrid,3,paris,4,nyc,6,miami
3675,750.0,1874.061677,ba8d60e9031abe641c8fcb5209cc0c5f,miami,12/01/2019,paris,4,madrid,3,nyc,6,miami
3712,972.0,2218.056889,07974516c323a35caacc39bda66d0052,miami,12/01/2019,nyc,6,paris,4,madrid,3,miami
2471,1126.0,2637.771610,8cf29a7fddcb556e4205288d2b779e45,miami,12/01/2019,nyc,6,madrid,3,paris,4,miami
525,1429.0,3224.287875,752515de0857c63004ee9879218ac388,miami,12/01/2019,paris,4,nyc,6,madrid,3,miami
6523,1436.0,3404.276434,4fa59484b20be4eed1e49f71ca660a33,miami,12/01/2019,madrid,3,nyc,6,paris,4,miami


In [ ]:
#make a list of columns name to transform
#loop through the column name and for each one, put inside the name of the column, df(column).to_datetime 